With tuning

In [7]:
#!/usr/bin/env python
# coding: utf-8

import os
import json
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from collections import defaultdict, Counter
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import LeaveOneOut, KFold, RandomizedSearchCV
from sklearn.metrics import mean_squared_error, r2_score, make_scorer
from scipy.stats import randint, uniform
import joblib

# ----------------- PATHS -----------------
INPUT_CSV = "/explore/nobackup/people/spotter5/new_combustion/2025-10-03_CombustionModelPredictors.csv"
OUT_DIR   = "/explore/nobackup/people/spotter5/new_combustion/LCC_new"
MODEL_DIR = os.path.join(OUT_DIR, "models")
os.makedirs(OUT_DIR, exist_ok=True)
os.makedirs(MODEL_DIR, exist_ok=True)

# ----------------- SEARCH CONFIG -----------------
RANDOM_STATE   = 42
N_JOBS         = -1
INNER_FOLDS    = 5
N_ITER_SEARCH  = 40
SCORER         = make_scorer(mean_squared_error, greater_is_better=False)

RF_PARAM_DIST = {
    "n_estimators": randint(200, 1000),
    "max_depth":    randint(3, 40),
    "max_features": uniform(0.2, 0.8),  # float in (0,1]: fraction of features
    "min_samples_split": randint(2, 20),
    "min_samples_leaf":  randint(1, 10),
    "bootstrap":   [True, False]
}

print(f"Reading: {INPUT_CSV}")
df = pd.read_csv(INPUT_CSV)
df = df[df['project'] .isin (['FISL', 'LC'])]

# ----------------- BASIC CLEANUP -----------------
df.columns = [c.strip() for c in df.columns]
rename_map = {}
if 'ID' in df.columns: rename_map['ID'] = 'id'
if 'Id' in df.columns: rename_map['Id'] = 'id'
if 'project_name' in df.columns and 'project.name' not in df.columns:
    rename_map['project_name'] = 'project.name'
if 'Date' in df.columns and 'date' not in df.columns:
    rename_map['Date'] = 'date'
if 'latitude' in df.columns and 'lat' not in df.columns:
    rename_map['latitude'] = 'lat'
if 'longitude' in df.columns and 'lon' not in df.columns:
    rename_map['longitude'] = 'lon'
if 'fireYr' in df.columns and 'burn_year' not in df.columns:
    rename_map['fireYr'] = 'burn_year'
df = df.rename(columns=rename_map)

# Schema snapshot
schema = pd.DataFrame({
    "column": df.columns,
    "dtype": df.dtypes.astype(str),
    "n_null": df.isna().sum(),
    "n_unique": [df[c].nunique(dropna=True) for c in df.columns]
})
schema.to_csv(os.path.join(OUT_DIR, "schema_summary.csv"), index=False)

# ----------------- CATEGORICAL: LandCover -> one-hot -----------------
if 'LandCover' in df.columns:
    df = pd.get_dummies(df, columns=['LandCover'], prefix='LC', drop_first=True, dummy_na=False)

# ----------------- EXCLUDED PREDICTOR COLUMNS -----------------
EXCLUDE_PRED_COLS = {
    'id', 'project.name', 'lat', 'lon', 'burn_year', 'date', 'project',
    'ID', 'Id', 'project_name', 'latitude', 'longitude', 'fireYr', 'Date',
    'landcover_name'
}

# ----------------- TARGET PICKER -----------------
def pick_col(candidates):
    for c in candidates:
        if c in df.columns:
            return c
    return None

# Try to detect column names for aboveground, belowground, and burn depth
COL_ABOVE = pick_col(['combusted_above', 'above.carbon.combusted'])
COL_BELOW = pick_col(['combusted_below'])
COL_DEPTH = pick_col(['burn_depth'])

# We will train ABOVEGROUND and BELOWGROUND combustion models now
targets = []
if COL_ABOVE:
    targets.append((COL_ABOVE, "kgC/m2"))   # adjust units as desired
if COL_BELOW:
    targets.append((COL_BELOW, "kgC/m2"))   # adjust units as desired

if not targets:
    raise ValueError("None of the expected aboveground or belowground target columns were found in the dataset.")

# IMPORTANT: include ALL possible combustion-related targets so they can be dropped from predictors
ALL_TARGET_COLS = [c for c in [COL_ABOVE, COL_BELOW, COL_DEPTH] if c]

# ----------------- Helper: build X, y -----------------
def build_xy(df_in: pd.DataFrame, target_col: str):
    # Drop obvious non-predictor columns
    drop_cols = [c for c in EXCLUDE_PRED_COLS if c in df_in.columns]
    work = df_in.drop(columns=drop_cols, errors='ignore').copy()

    # Keep only rows with non-NaN target
    work = work.dropna(subset=[target_col])

    y = work[target_col].astype(float).copy()

    # Drop the explicit target AND any other known combustion targets (above, below, depth)
    X = work.drop(columns=list(set(ALL_TARGET_COLS + [target_col])), errors='ignore')

    # Guard against accidental leakage of any combustion target
    leak_cols = [c for c in [COL_ABOVE, COL_BELOW, COL_DEPTH] if c]
    for lc in leak_cols:
        assert lc not in X.columns, f"Target leakage: {lc} present in predictors!"

    non_numeric = X.select_dtypes(exclude=[np.number]).columns.tolist()
    if non_numeric:
        X = X.drop(columns=non_numeric)

    # Sanity: no empty feature set
    if X.shape[1] == 0:
        raise ValueError(f"No numeric predictors left after preprocessing for target '{target_col}'.")
    return X, y

# ----------------- RandomizedSearch + LOOCV with per-fold R² -----------------
def run_target_randomsearch_loocv(target_col: str, units_label: str = "units"):
    X, y = build_xy(df, target_col)
    if X.shape[1] == 0 or len(y) < 3:
        print(f"[ERROR] Not enough predictors or samples for '{target_col}'.")
        return

    print(f"\nTarget: {target_col} | X: {X.shape} | y: {y.shape}")
    y = y.reset_index(drop=True)
    X = X.reset_index(drop=True)
    out_prefix = target_col.replace('.', '_')

    # 1. RandomizedSearchCV (global)
    print(f"\n[{target_col}] Starting global RandomizedSearchCV with {N_ITER_SEARCH} iterations...")
    kfold = KFold(n_splits=min(INNER_FOLDS, len(y)), shuffle=True, random_state=RANDOM_STATE)
    base = RandomForestRegressor(random_state=RANDOM_STATE, n_jobs=N_JOBS)
    tuner = RandomizedSearchCV(
        estimator=base,
        param_distributions=RF_PARAM_DIST,
        n_iter=N_ITER_SEARCH,
        scoring=SCORER,
        cv=kfold,
        random_state=RANDOM_STATE,
        n_jobs=N_JOBS,
        verbose=1,
        refit=True,
    )
    tuner.fit(X, y)

    # Leakage sanity check: the tuned model must NOT expect any combustion target as a feature
    tuned_features = list(getattr(tuner.best_estimator_, "feature_names_in_", []))
    leak_cols = [c for c in [COL_ABOVE, COL_BELOW, COL_DEPTH] if c]
    for lc in leak_cols:
        if lc in tuned_features:
            raise RuntimeError(
                f"Leakage detected: tuned model expects combustion target '{lc}' as a feature. "
                f"Check preprocessing."
            )

    best_params = tuner.best_params_
    best_neg_mse = float(tuner.best_score_)
    best_rmse = float(np.sqrt(-best_neg_mse))
    print(f"[{target_col}] Best params: {best_params}")
    print(f"[{target_col}] RandomizedSearch best CV RMSE: {best_rmse:.4f} {units_label}")
    rs_results = pd.DataFrame(tuner.cv_results_)
    rs_results.to_csv(os.path.join(OUT_DIR, f"{out_prefix}_random_search_results.csv"), index=False)

    # 2. LOOCV using best params
    print(f"\n[{target_col}] Starting LOOCV with fixed best hyperparameters...")
    loo = LeaveOneOut()
    n = len(y)
    y_pred = np.zeros(n, dtype=float)
    r2_folds = np.full(n, np.nan, dtype=float)  # one R² per left-out observation
    train_means = np.full(n, np.nan, dtype=float)

    for i, (train_idx, test_idx) in enumerate(loo.split(X), start=1):
        Xtr, Xte = X.iloc[train_idx], X.iloc[test_idx]
        ytr = y.iloc[train_idx]
        yte = y.iloc[test_idx].values[0]

        model = RandomForestRegressor(random_state=RANDOM_STATE, n_jobs=N_JOBS, **best_params)
        model.fit(Xtr, ytr)
        yhat = model.predict(Xte)[0]

        # store prediction
        test_i = test_idx[0]
        y_pred[test_i] = yhat

        # per-fold R² for this left-out sample
        mu_train = float(ytr.mean())
        train_means[test_i] = mu_train
        denom = (yte - mu_train)**2
        num = (yte - yhat)**2

        # if denom == 0, R² is undefined -> keep NaN
        if denom != 0.0:
            r2_val = 1.0 - (num / denom)
            # clamp negative R² to 0
            if r2_val < 0.0:
                r2_val = 0.0
            r2_folds[test_i] = r2_val

        if i % 25 == 0 or i == n:
            print(f"  LOOCV progress: {i}/{n}")

    # Global LOOCV metrics over all predictions
    rmse_global = mean_squared_error(y, y_pred, squared=False)
    r2_raw_global = r2_score(y, y_pred)
    # clamp negative global R² to 0 as well
    r2_global = max(0.0, r2_raw_global)
    print(f"[{target_col}] LOOCV RMSE (global): {rmse_global:.4f} | R² (global, clamped): {r2_global:.4f} (raw: {r2_raw_global:.4f})")

    # Save per-sample predictions + per-fold R²
    preds_df = pd.DataFrame({
        "index": np.arange(n),
        "y_obs": y.values,
        "y_pred": y_pred,
        "train_mean_y": train_means,
        "r2_loocv_fold": r2_folds
    })

    # CSV for violin plot
    violin_csv_path = os.path.join(OUT_DIR, f"{out_prefix}_violin.csv")
    preds_df.to_csv(violin_csv_path, index=False)
    print(f"[{target_col}] Saved per-fold R² CSV: {violin_csv_path}")

    # Also keep the LOOCV predictions CSV
    preds_path = os.path.join(OUT_DIR, f"{out_prefix}_loocv_predictions.csv")
    preds_df.to_csv(preds_path, index=False)
    print(f"[{target_col}] Saved LOOCV predictions (with fold R²) to: {preds_path}")

    # Save summary metrics
    pd.DataFrame({
        "target": [target_col],
        "n": [n],
        "n_predictors": [X.shape[1]],
        "loocv_rmse_global": [rmse_global],
        "loocv_r2_global_clamped": [r2_global],
        "loocv_r2_global_raw": [r2_raw_global],
        "random_search_best_rmse": [best_rmse]
    }).to_csv(os.path.join(OUT_DIR, f"{out_prefix}_loocv_metrics.csv"), index=False)

    # Plot 1:1 scatter (global)
    plt.figure(figsize=(8, 6))
    sns.scatterplot(x=preds_df["y_obs"], y=preds_df["y_pred"], s=18, edgecolor=None)
    lo = float(np.nanmin([preds_df["y_obs"].min(), preds_df["y_pred"].min()]))
    hi = float(np.nanmax([preds_df["y_obs"].max(), preds_df["y_pred"].max()]))
    plt.plot([lo, hi], [lo, hi], 'k--', lw=2)
    plt.xlabel(f"Observed {target_col}")
    plt.ylabel(f"Predicted {target_col}")
    plt.title(f"{target_col}: LOOCV Obs vs Pred\nRMSE={rmse_global:.3f}, R²={r2_global:.3f} (clamped)")
    plt.grid(True)
    plt.tight_layout()
    plt.savefig(os.path.join(OUT_DIR, f"{out_prefix}_loocv_obs_pred.png"), dpi=150)
    plt.close()

    # ----------------- VIOLIN PLOT OF PER-FOLD R² -----------------
    valid_r2 = preds_df["r2_loocv_fold"].dropna()
    if len(valid_r2) > 0:
        plt.figure(figsize=(6, 6))
        sns.violinplot(y=valid_r2, cut=0)
        plt.ylabel("Per-fold LOOCV R²")
        plt.title(f"Distribution of LOOCV R² per left-out sample\nTarget: {target_col}")
        plt.grid(axis='y', alpha=0.3)
        plt.tight_layout()
        violin_png_path = os.path.join(OUT_DIR, f"{out_prefix}_violin.png")
        plt.savefig(violin_png_path, dpi=150)
        plt.close()
        print(f"[{target_col}] Saved violin plot of per-fold R²: {violin_png_path}")
    else:
        print(f"[{target_col}] No valid per-fold R² values (all denominators zero). Skipping violin plot.")

    # Save model + metadata (with feature names!)
    final_model = RandomForestRegressor(random_state=RANDOM_STATE, n_jobs=N_JOBS, **best_params)
    final_model.fit(X, y)
    model_path = os.path.join(MODEL_DIR, f"rf_final_{out_prefix}.joblib")
    joblib.dump(final_model, model_path)

    feature_names = list(getattr(final_model, "feature_names_in_", X.columns))

    # --------------- NEW: FEATURE IMPORTANCE CSV + PLOT ---------------
    importances = final_model.feature_importances_
    fi_df = pd.DataFrame({
        "feature": feature_names,
        "importance": importances
    }).sort_values("importance", ascending=False)

    fi_csv_path = os.path.join(OUT_DIR, f"{out_prefix}_feature_importance.csv")
    fi_df.to_csv(fi_csv_path, index=False)
    print(f"[{target_col}] Saved feature importance CSV: {fi_csv_path}")

    # Barplot of top N features
    top_n = min(30, len(fi_df))
    fi_top = fi_df.head(top_n)

    plt.figure(figsize=(10, max(6, 0.3 * top_n)))
    sns.barplot(data=fi_top, x="importance", y="feature", orient="h")
    plt.xlabel("Random Forest feature importance")
    plt.ylabel("Feature")
    plt.title(f"Feature importance (top {top_n})\nTarget: {target_col}")
    plt.tight_layout()
    fi_png_path = os.path.join(OUT_DIR, f"{out_prefix}_feature_importance.png")
    plt.savefig(fi_png_path, dpi=150)
    plt.close()
    print(f"[{target_col}] Saved feature importance plot: {fi_png_path}")
    # --------------- END NEW BLOCK -----------------

    meta = {
        "target": target_col,
        "loocv_rmse_global": rmse_global,
        "loocv_r2_global_clamped": r2_global,
        "loocv_r2_global_raw": r2_raw_global,
        "best_params": best_params,
        "model_path": model_path,
        "n_samples": n,
        "feature_names": feature_names
    }
    with open(os.path.join(OUT_DIR, f"{out_prefix}_final_model_metadata.json"), "w") as f:
        json.dump(meta, f, indent=2)

    # Extra sanity message
    print(f"[{target_col}] Final model trained with {len(meta['feature_names'])} features; "
          f"'{target_col}' in features? {target_col in meta['feature_names']}")

# ----------------- RUN FOR EACH TARGET (ABOVEGROUND, BELOWGROUND) -----------------
for tcol, units in targets:
    run_target_randomsearch_loocv(tcol, units)

print("\nDone.")


Reading: /explore/nobackup/people/spotter5/new_combustion/2025-10-03_CombustionModelPredictors.csv

Target: combusted_above | X: (270, 50) | y: (270,)

[combusted_above] Starting global RandomizedSearchCV with 40 iterations...
Fitting 5 folds for each of 40 candidates, totalling 200 fits
[combusted_above] Best params: {'bootstrap': True, 'max_depth': 19, 'max_features': 0.413424811420228, 'min_samples_leaf': 2, 'min_samples_split': 3, 'n_estimators': 419}
[combusted_above] RandomizedSearch best CV RMSE: 402.8719 kgC/m2

[combusted_above] Starting LOOCV with fixed best hyperparameters...
  LOOCV progress: 25/270
  LOOCV progress: 50/270
  LOOCV progress: 75/270
  LOOCV progress: 100/270
  LOOCV progress: 125/270
  LOOCV progress: 150/270
  LOOCV progress: 175/270
  LOOCV progress: 200/270
  LOOCV progress: 225/270
  LOOCV progress: 250/270
  LOOCV progress: 270/270
[combusted_above] LOOCV RMSE (global): 387.2333 | R² (global, clamped): 0.2224 (raw: 0.2224)
[combusted_above] Saved per-fo

Just depth

In [8]:
#!/usr/bin/env python
# coding: utf-8

import os
import json
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from collections import defaultdict, Counter
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import LeaveOneOut, KFold, RandomizedSearchCV
from sklearn.metrics import mean_squared_error, r2_score, make_scorer
from scipy.stats import randint, uniform
import joblib

# ----------------- PATHS -----------------
INPUT_CSV = "/explore/nobackup/people/spotter5/new_combustion/2025-10-03_CombustionModelPredictors.csv"
OUT_DIR   = "/explore/nobackup/people/spotter5/new_combustion/LCC_new"
MODEL_DIR = os.path.join(OUT_DIR, "models")
os.makedirs(OUT_DIR, exist_ok=True)
os.makedirs(MODEL_DIR, exist_ok=True)

# ----------------- SEARCH CONFIG -----------------
RANDOM_STATE   = 42
N_JOBS         = -1
INNER_FOLDS    = 5
N_ITER_SEARCH  = 40
SCORER         = make_scorer(mean_squared_error, greater_is_better=False)

RF_PARAM_DIST = {
    "n_estimators": randint(200, 1000),
    "max_depth":    randint(3, 40),
    "max_features": uniform(0.2, 0.8),  # float in (0,1]: fraction of features
    "min_samples_split": randint(2, 20),
    "min_samples_leaf":  randint(1, 10),
    "bootstrap":   [True, False]
}

print(f"Reading: {INPUT_CSV}")
df = pd.read_csv(INPUT_CSV)
df = df[df['project'] .isin (['FISL', 'LC'])]

# ----------------- BASIC CLEANUP -----------------
df.columns = [c.strip() for c in df.columns]
rename_map = {}
if 'ID' in df.columns: rename_map['ID'] = 'id'
if 'Id' in df.columns: rename_map['Id'] = 'id'
if 'project_name' in df.columns and 'project.name' not in df.columns:
    rename_map['project_name'] = 'project.name'
if 'Date' in df.columns and 'date' not in df.columns:
    rename_map['Date'] = 'date'
if 'latitude' in df.columns and 'lat' not in df.columns:
    rename_map['latitude'] = 'lat'
if 'longitude' in df.columns and 'lon' not in df.columns:
    rename_map['longitude'] = 'lon'
if 'fireYr' in df.columns and 'burn_year' not in df.columns:
    rename_map['fireYr'] = 'burn_year'
df = df.rename(columns=rename_map)

# Schema snapshot
schema = pd.DataFrame({
    "column": df.columns,
    "dtype": df.dtypes.astype(str),
    "n_null": df.isna().sum(),
    "n_unique": [df[c].nunique(dropna=True) for c in df.columns]
})
schema.to_csv(os.path.join(OUT_DIR, "schema_summary.csv"), index=False)

# ----------------- CATEGORICAL: LandCover -> one-hot -----------------
if 'LandCover' in df.columns:
    df = pd.get_dummies(df, columns=['LandCover'], prefix='LC', drop_first=True, dummy_na=False)

# ----------------- EXCLUDED PREDICTOR COLUMNS -----------------
EXCLUDE_PRED_COLS = {
    'id', 'project.name', 'lat', 'lon', 'burn_year', 'date', 'project',
    'ID', 'Id', 'project_name', 'latitude', 'longitude', 'fireYr', 'Date',
    'landcover_name'
}

# ----------------- TARGET PICKER -----------------
def pick_col(candidates):
    for c in candidates:
        if c in df.columns:
            return c
    return None

COL_ABOVE = pick_col(['combusted_above', 'above.carbon.combusted'])
COL_BELOW = pick_col(['combusted_below'])
COL_DEPTH = pick_col(['burn_depth'])

# >>> Train burn_depth only for now <<<
targets = [(c, "units") for c in [COL_DEPTH] if c]
if not targets:
    raise ValueError("None of the expected target columns were found in the dataset.")

# IMPORTANT: include ALL possible targets for safe dropping
ALL_TARGET_COLS = [c for c in [COL_ABOVE, COL_BELOW, COL_DEPTH] if c]

# ----------------- Helper: build X, y -----------------
def build_xy(df_in: pd.DataFrame, target_col: str):
    drop_cols = [c for c in EXCLUDE_PRED_COLS if c in df_in.columns]
    work = df_in.drop(columns=drop_cols, errors='ignore').copy()
    work = work.dropna(subset=[target_col])

    y = work[target_col].astype(float).copy()

    # Drop the explicit target AND any other known target columns to prevent leakage
    X = work.drop(columns=list(set(ALL_TARGET_COLS + [target_col])), errors='ignore')

    # Guard against accidental leakage
    assert target_col not in X.columns, f"Target leakage: {target_col} present in predictors!"

    non_numeric = X.select_dtypes(exclude=[np.number]).columns.tolist()
    if non_numeric:
        X = X.drop(columns=non_numeric)

    # Sanity: no empty feature set
    if X.shape[1] == 0:
        raise ValueError(f"No numeric predictors left after preprocessing for target '{target_col}'.")
    return X, y

# ----------------- RandomizedSearch + LOOCV with per-fold R² -----------------
def run_target_randomsearch_loocv(target_col: str, units_label: str = "units"):
    X, y = build_xy(df, target_col)
    if X.shape[1] == 0 or len(y) < 3:
        print(f"[ERROR] Not enough predictors or samples for '{target_col}'.")
        return

    print(f"\nTarget: {target_col} | X: {X.shape} | y: {y.shape}")
    y = y.reset_index(drop=True)
    X = X.reset_index(drop=True)
    out_prefix = target_col.replace('.', '_')

    # 1. RandomizedSearchCV (global)
    print(f"\n[{target_col}] Starting global RandomizedSearchCV with {N_ITER_SEARCH} iterations...")
    kfold = KFold(n_splits=min(INNER_FOLDS, len(y)), shuffle=True, random_state=RANDOM_STATE)
    base = RandomForestRegressor(random_state=RANDOM_STATE, n_jobs=N_JOBS)
    tuner = RandomizedSearchCV(
        estimator=base,
        param_distributions=RF_PARAM_DIST,
        n_iter=N_ITER_SEARCH,
        scoring=SCORER,
        cv=kfold,
        random_state=RANDOM_STATE,
        n_jobs=N_JOBS,
        verbose=1,
        refit=True,
    )
    tuner.fit(X, y)

    # Leakage sanity check: the tuned model must NOT expect the target as a feature
    tuned_features = list(getattr(tuner.best_estimator_, "feature_names_in_", []))
    if target_col in tuned_features:
        raise RuntimeError(
            f"Leakage detected: tuned model expects target '{target_col}' as a feature. "
            f"Check preprocessing."
        )

    best_params = tuner.best_params_
    best_neg_mse = float(tuner.best_score_)
    best_rmse = float(np.sqrt(-best_neg_mse))
    print(f"[{target_col}] Best params: {best_params}")
    print(f"[{target_col}] RandomizedSearch best CV RMSE: {best_rmse:.4f} {units_label}")
    rs_results = pd.DataFrame(tuner.cv_results_)
    rs_results.to_csv(os.path.join(OUT_DIR, f"{out_prefix}_random_search_results.csv"), index=False)

    # 2. LOOCV using best params
    print(f"\n[{target_col}] Starting LOOCV with fixed best hyperparameters...")
    loo = LeaveOneOut()
    n = len(y)
    y_pred = np.zeros(n, dtype=float)
    r2_folds = np.full(n, np.nan, dtype=float)  # one R² per left-out observation
    train_means = np.full(n, np.nan, dtype=float)

    for i, (train_idx, test_idx) in enumerate(loo.split(X), start=1):
        Xtr, Xte = X.iloc[train_idx], X.iloc[test_idx]
        ytr = y.iloc[train_idx]
        yte = y.iloc[test_idx].values[0]

        model = RandomForestRegressor(random_state=RANDOM_STATE, n_jobs=N_JOBS, **best_params)
        model.fit(Xtr, ytr)
        yhat = model.predict(Xte)[0]

        # store prediction
        test_i = test_idx[0]
        y_pred[test_i] = yhat

        # per-fold R² for this left-out sample
        mu_train = float(ytr.mean())
        train_means[test_i] = mu_train
        denom = (yte - mu_train)**2
        num = (yte - yhat)**2

        # if denom == 0, R² is undefined -> keep NaN
        if denom != 0.0:
            r2_val = 1.0 - (num / denom)
            # clamp negative R² to 0
            if r2_val < 0.0:
                r2_val = 0.0
            r2_folds[test_i] = r2_val

        if i % 25 == 0 or i == n:
            print(f"  LOOCV progress: {i}/{n}")

    # Global LOOCV metrics over all predictions
    rmse_global = mean_squared_error(y, y_pred, squared=False)
    r2_raw_global = r2_score(y, y_pred)
    # clamp negative global R² to 0 as well
    r2_global = max(0.0, r2_raw_global)
    print(f"[{target_col}] LOOCV RMSE (global): {rmse_global:.4f} | R² (global, clamped): {r2_global:.4f} (raw: {r2_raw_global:.4f})")

    # Save per-sample predictions + per-fold R²
    preds_df = pd.DataFrame({
        "index": np.arange(n),
        "y_obs": y.values,
        "y_pred": y_pred,
        "train_mean_y": train_means,
        "r2_loocv_fold": r2_folds
    })

    # CSV for violin plot
    violin_csv_path = os.path.join(OUT_DIR, f"{out_prefix}_violin.csv")
    preds_df.to_csv(violin_csv_path, index=False)
    print(f"[{target_col}] Saved per-fold R² CSV: {violin_csv_path}")

    # Also keep the LOOCV predictions CSV
    preds_path = os.path.join(OUT_DIR, f"{out_prefix}_loocv_predictions.csv")
    preds_df.to_csv(preds_path, index=False)
    print(f"[{target_col}] Saved LOOCV predictions (with fold R²) to: {preds_path}")

    # Save summary metrics
    pd.DataFrame({
        "target": [target_col],
        "n": [n],
        "n_predictors": [X.shape[1]],
        "loocv_rmse_global": [rmse_global],
        "loocv_r2_global_clamped": [r2_global],
        "loocv_r2_global_raw": [r2_raw_global],
        "random_search_best_rmse": [best_rmse]
    }).to_csv(os.path.join(OUT_DIR, f"{out_prefix}_loocv_metrics.csv"), index=False)

    # Plot 1:1 scatter (global)
    plt.figure(figsize=(8, 6))
    sns.scatterplot(x=preds_df["y_obs"], y=preds_df["y_pred"], s=18, edgecolor=None)
    lo = float(np.nanmin([preds_df["y_obs"].min(), preds_df["y_pred"].min()]))
    hi = float(np.nanmax([preds_df["y_obs"].max(), preds_df["y_pred"].max()]))
    plt.plot([lo, hi], [lo, hi], 'k--', lw=2)
    plt.xlabel(f"Observed {target_col}")
    plt.ylabel(f"Predicted {target_col}")
    plt.title(f"{target_col}: LOOCV Obs vs Pred\nRMSE={rmse_global:.3f}, R²={r2_global:.3f} (clamped)")
    plt.grid(True)
    plt.tight_layout()
    plt.savefig(os.path.join(OUT_DIR, f"{out_prefix}_loocv_obs_pred.png"), dpi=150)
    plt.close()

    # ----------------- VIOLIN PLOT OF PER-FOLD R² -----------------
    valid_r2 = preds_df["r2_loocv_fold"].dropna()
    if len(valid_r2) > 0:
        plt.figure(figsize=(6, 6))
        sns.violinplot(y=valid_r2, cut=0)
        plt.ylabel("Per-fold LOOCV R²")
        plt.title(f"Distribution of LOOCV R² per left-out sample\nTarget: {target_col}")
        plt.grid(axis='y', alpha=0.3)
        plt.tight_layout()
        violin_png_path = os.path.join(OUT_DIR, f"{out_prefix}_violin.png")
        plt.savefig(violin_png_path, dpi=150)
        plt.close()
        print(f"[{target_col}] Saved violin plot of per-fold R²: {violin_png_path}")
    else:
        print(f"[{target_col}] No valid per-fold R² values (all denominators zero). Skipping violin plot.")

    # Save model + metadata (with feature names!)
    final_model = RandomForestRegressor(random_state=RANDOM_STATE, n_jobs=N_JOBS, **best_params)
    final_model.fit(X, y)
    model_path = os.path.join(MODEL_DIR, f"rf_final_{out_prefix}.joblib")
    joblib.dump(final_model, model_path)

    feature_names = list(getattr(final_model, "feature_names_in_", X.columns))

    # --------------- FEATURE IMPORTANCE CSV + PLOT ---------------
    importances = final_model.feature_importances_
    fi_df = pd.DataFrame({
        "feature": feature_names,
        "importance": importances
    }).sort_values("importance", ascending=False)

    fi_csv_path = os.path.join(OUT_DIR, f"{out_prefix}_feature_importance.csv")
    fi_df.to_csv(fi_csv_path, index=False)
    print(f"[{target_col}] Saved feature importance CSV: {fi_csv_path}")

    top_n = min(30, len(fi_df))
    fi_top = fi_df.head(top_n)

    plt.figure(figsize=(10, max(6, 0.3 * top_n)))
    sns.barplot(data=fi_top, x="importance", y="feature", orient="h")
    plt.xlabel("Random Forest feature importance")
    plt.ylabel("Feature")
    plt.title(f"Feature importance (top {top_n})\nTarget: {target_col}")
    plt.tight_layout()
    fi_png_path = os.path.join(OUT_DIR, f"{out_prefix}_feature_importance.png")
    plt.savefig(fi_png_path, dpi=150)
    plt.close()
    print(f"[{target_col}] Saved feature importance plot: {fi_png_path}")
    # ------------- END FEATURE IMPORTANCE BLOCK -------------

    meta = {
        "target": target_col,
        "loocv_rmse_global": rmse_global,
        "loocv_r2_global_clamped": r2_global,
        "loocv_r2_global_raw": r2_raw_global,
        "best_params": best_params,
        "model_path": model_path,
        "n_samples": n,
        "feature_names": feature_names
    }
    with open(os.path.join(OUT_DIR, f"{out_prefix}_final_model_metadata.json"), "w") as f:
        json.dump(meta, f, indent=2)

    # Extra sanity message
    print(f"[{target_col}] Final model trained with {len(meta['feature_names'])} features; "
          f"'{target_col}' in features? {target_col in meta['feature_names']}")

# ----------------- RUN FOR EACH TARGET -----------------
for tcol, units in targets:
    run_target_randomsearch_loocv(tcol, units)

print("\nDone.")


Reading: /explore/nobackup/people/spotter5/new_combustion/2025-10-03_CombustionModelPredictors.csv

Target: burn_depth | X: (270, 50) | y: (270,)

[burn_depth] Starting global RandomizedSearchCV with 40 iterations...
Fitting 5 folds for each of 40 candidates, totalling 200 fits
[burn_depth] Best params: {'bootstrap': True, 'max_depth': 14, 'max_features': 0.40142583666029136, 'min_samples_leaf': 2, 'min_samples_split': 4, 'n_estimators': 312}
[burn_depth] RandomizedSearch best CV RMSE: 5.3291 units

[burn_depth] Starting LOOCV with fixed best hyperparameters...
  LOOCV progress: 25/270
  LOOCV progress: 50/270
  LOOCV progress: 75/270
  LOOCV progress: 100/270
  LOOCV progress: 125/270
  LOOCV progress: 150/270
  LOOCV progress: 175/270
  LOOCV progress: 200/270
  LOOCV progress: 225/270
  LOOCV progress: 250/270
  LOOCV progress: 270/270
[burn_depth] LOOCV RMSE (global): 4.8438 | R² (global, clamped): 0.3910 (raw: 0.3910)
[burn_depth] Saved per-fold R² CSV: /explore/nobackup/people/sp

In [11]:
't'

't'

Xgboost with tuning